In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from astropy.table import vstack
from scipy.optimize import leastsq, minimize

from scipy.stats import chisquare
import scienceplots
#from photometric_models import HG, HG12, HG1G2
from db import query
#import emcee
#import corner
from tqdm import tqdm
import time

plt.style.use('science')

filters = ['g', 'r', 'i', 'z']
filter_colors = {'g': '#008060', 'r': '#ff4000', 'i': '#850000', 'z': '#6600cc'}
linestyles = {'g': "--", "r":"-.", "i":":", "z":"-"}
regions = {
    "NEO": "q <= 1.3",
    "Main Belt": "(mpc.e <1) and (mpc.q/(1-mpc.e) >= 1.6) and (mpc.q/(1-mpc.e) <= 4.2 and mpc.q>=1.3)",
    "Jupiter Trojans": "(mpc.q/(1-mpc.e) >= 5.05) and (mpc.q/(1-mpc.e) <= 5.35)" #5.05 and 5.35 AU
}

attribs = ['Chi2', 'G12', 'G12Err', 'H', 'HErr', 'Ndata']

NEOs must have N = 25 and 10 points in the moderate coverage range

In [9]:
#photo_table.to_pandas().to_csv("./photometric/neo.csv")
df_neo = pd.read_csv("./phase_parameters/neo.csv", index_col=False)
'''df_photo = pd.read_csv("./photometric/neo.csv", index_col=False)
df_merge = pd.merge(left=df, right= df_photo, on="ssObjectId")'''

df_neo

,Unnamed: 0,ssObjectId,band,count,minPhase,maxPhase,phaseDensity,oppCOV,modCOV,highCOV,avgMag,magSTD,avgMagErr,MagErrSTD
0,0,-9222952578217572358,g,4,16.825115,33.296112,4.117749,0,0,4,22.778250,0.376740,0.077000,0.031443
1,1,-9222952578217572358,i,11,15.710444,49.469246,3.068982,0,0,11,22.177182,1.396005,0.219818,0.501258
2,2,-9222952578217572358,r,12,15.307009,49.445053,2.844837,0,0,12,23.206167,4.047751,0.487333,1.501578
3,3,-9222952578217572358,z,6,21.359219,37.461310,2.683682,0,0,6,23.393333,4.317069,0.538167,1.129170
4,4,-9222809672762914664,g,3,15.760029,16.081547,0.107173,0,0,3,29.653334,3.346718,5.326667,0.366618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155777,155777,9218400096519481639,z,11,5.977979,59.565080,4.871555,0,5,6,21.604636,2.267825,0.232364,0.537087
155778,155778,9222765823057774610,g,2,10.795681,16.813921,3.009120,0,1,1,28.816500,1.007627,3.510500,0.850650
155779,155779,9222765823057774610,i,15,5.481915,20.515633,1.002248,0,12,3,27.851667,2.597210,3.175133,0.583045
155780,155780,9222765823057774610,r,11,7.125856,28.669268,1.958492,0,6,5,29.430727,3.260058,3.029818,0.479445


In [6]:
df_neo[(df_neo["count"] >= 25) & (df_neo["modCOV"] >= 10)]

,Unnamed: 0,ssObjectId,band,count,minPhase,maxPhase,phaseDensity,oppCOV,modCOV,highCOV,avgMag,magSTD,avgMagErr,MagErrSTD
37,37,-9208655856804061175,i,37,1.214715,32.357777,0.841704,4,16,17,22.488324,0.714719,0.116649,0.142760
38,38,-9208655856804061175,r,38,0.822222,42.395546,1.094035,2,17,19,22.856079,0.799087,0.122974,0.137558
39,39,-9208655856804061175,z,28,0.835593,29.339643,1.018002,6,13,9,22.161679,0.533688,0.136786,0.093330
60,60,-9199351727774599856,g,30,7.991679,30.945797,0.765137,0,13,17,23.054667,0.533968,0.073133,0.034886
61,61,-9199351727774599856,i,70,8.024878,35.361378,0.390521,0,24,46,22.368543,0.535241,0.095471,0.050737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155631,155631,9155663528343865287,i,28,1.898554,40.165730,1.366685,1,16,11,24.332643,4.448340,1.203286,1.827254
155632,155632,9155663528343865287,r,28,1.124453,40.160824,1.394156,1,15,12,22.831857,1.846019,0.294214,0.651708
155653,155653,9169672633601948315,z,76,4.867282,62.202526,0.754411,0,11,65,20.172316,1.733893,0.056526,0.053624
155664,155664,9180800238072549959,r,27,8.645984,44.133060,1.314336,0,11,16,23.395333,0.537920,0.149704,0.064512


In [8]:
# Pull samples out and check them
# Can I check how well these curves are?

In [2]:
# For 10 years

Main belt asteroids must have N = 30 and 5 points in low to model opposition effect, 10 points within moderate, 5 points in high to model the non-linearities at alpah > 15.

In [10]:
#photo_table.to_pandas().to_csv("./photometric/neo.csv")
df_mb = pd.read_csv("./phase_parameters/main_belt.csv", index_col=False)
'''df_photo = pd.read_csv("./photometric/neo.csv", index_col=False)
df_merge = pd.merge(left=df, right= df_photo, on="ssObjectId")'''

df_mb

,Unnamed: 0,ssObjectId,band,count,minPhase,maxPhase,phaseDensity,oppCOV,modCOV,highCOV,avgMag,magSTD,avgMagErr,MagErrSTD
0,0,-9222239669212729645,g,19,6.182309,30.894860,1.300661,0,10,9,21.977737,0.722553,0.031368,0.023445
1,1,-9222239669212729645,i,62,5.803712,31.713652,0.417902,0,28,34,21.509984,0.714302,0.053597,0.033487
2,2,-9222239669212729645,r,56,6.178031,31.713594,0.455992,0,16,40,21.618143,0.802314,0.035589,0.024218
3,3,-9222239669212729645,z,48,5.805296,31.536995,0.536077,0,26,22,21.602125,0.497391,0.080854,0.036883
4,4,-9221730060911893173,g,9,1.621336,19.365707,1.971597,1,4,4,24.467222,0.433258,0.212889,0.093637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16906792,16906792,9221766284933543335,z,1,4.853111,4.853111,0.000000,0,1,0,22.621000,NaN,0.207000,NaN
16906793,16906793,9223366833108713925,g,16,2.732214,26.177430,1.465326,0,14,2,23.973688,0.475322,0.180375,0.140484
16906794,16906794,9223366833108713925,i,17,5.572031,25.864360,1.193666,0,12,5,23.587588,0.859114,0.277706,0.302998
16906795,16906795,9223366833108713925,r,20,2.740132,22.556175,0.990802,0,15,5,23.726900,0.860614,0.258000,0.315224


In [12]:
df_mb[(df_mb["count"] >= 30) & (df_mb["modCOV"] >= 10) & (df_mb["highCOV"] >= 5)]

,Unnamed: 0,ssObjectId,band,count,minPhase,maxPhase,phaseDensity,oppCOV,modCOV,highCOV,avgMag,magSTD,avgMagErr,MagErrSTD
1,1,-9222239669212729645,i,62,5.803712,31.713652,0.417902,0,28,34,21.509984,0.714302,0.053597,0.033487
2,2,-9222239669212729645,r,56,6.178031,31.713594,0.455992,0,16,40,21.618143,0.802314,0.035589,0.024218
3,3,-9222239669212729645,z,48,5.805296,31.536995,0.536077,0,26,22,21.602125,0.497391,0.080854,0.036883
9,9,-9217183400242351593,i,63,1.577819,30.358425,0.456835,1,29,33,22.115778,0.675622,0.091810,0.057458
10,10,-9217183400242351593,r,63,1.165873,29.918692,0.456394,5,20,38,22.512809,0.687310,0.086270,0.049517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16906782,16906782,9221143240477431985,i,55,2.392312,18.447186,0.291907,0,46,9,22.965909,0.402393,0.146564,0.087332
16906783,16906783,9221143240477431985,r,76,4.755475,18.678936,0.183203,0,60,16,23.179961,0.419858,0.109303,0.041382
16906786,16906786,9221737176446518823,i,82,2.005150,21.601088,0.238975,0,57,25,20.494976,0.825260,0.027732,0.029537
16906787,16906787,9221737176446518823,r,80,0.301994,21.601116,0.266239,4,54,22,20.714363,0.847297,0.024425,0.031069


Jupiter Trojans must have N = 20, with $N_{opp}$ > 5

In [14]:
df_jt = pd.read_csv("./phase_parameters/jupiter_trojans.csv", index_col=False)
df_jt

,Unnamed: 0,ssObjectId,band,count,minPhase,maxPhase,phaseDensity,oppCOV,modCOV,highCOV,avgMag,magSTD,avgMagErr,MagErrSTD
0,0,-9223228083474041721,g,27,0.665575,10.687481,0.371182,2,25,0,22.798074,0.296232,0.060556,0.022541
1,1,-9223228083474041721,i,52,4.109375,10.909799,0.130777,0,52,0,22.355192,0.286567,0.097865,0.048217
2,2,-9223228083474041721,r,62,0.859652,11.532287,0.172139,2,60,0,22.444000,0.264899,0.068694,0.026121
3,3,-9223228083474041721,z,38,5.876497,10.678315,0.126364,0,38,0,22.300000,0.267190,0.145737,0.040181
4,4,-9222033681059432591,g,28,2.867289,12.188930,0.332916,0,28,0,22.551000,0.343431,0.049107,0.020220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355280,355280,9222169029913136011,z,3,4.669189,7.696556,1.009122,0,3,0,23.365334,0.290538,0.276333,0.032005
355281,355281,9222717556902754603,g,52,0.825239,11.409647,0.203546,12,40,0,22.548481,0.308372,0.054712,0.027233
355282,355282,9222717556902754603,i,107,0.255454,11.465165,0.104764,11,96,0,22.074598,0.323386,0.077421,0.034290
355283,355283,9222717556902754603,r,122,0.266466,11.409750,0.091338,15,107,0,22.209025,0.370042,0.061254,0.042517


In [16]:
df_jt[(df_mb["count"] >= 20) & (df_mb["oppCOV"] >= 5)]

/var/folders/44/yjf9jwgd1bqfct8dx076vwqm0000gn/T/ipykernel_13129/2560279063.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_jt[(df_mb["count"] >= 20) & (df_mb["oppCOV"] >= 5)]


,Unnamed: 0,ssObjectId,band,count,minPhase,maxPhase,phaseDensity,oppCOV,modCOV,highCOV,avgMag,magSTD,avgMagErr,MagErrSTD
10,10,-9221509829191769804,r,31,1.999048,10.406020,0.271193,1,30,0,23.793645,0.299076,0.157484,0.048550
13,13,-9216338841535073204,i,70,0.903671,12.189437,0.161225,2,68,0,22.848157,0.421439,0.126657,0.044234
14,14,-9216338841535073204,r,88,0.909049,12.189139,0.128183,2,86,0,23.106636,0.403522,0.104261,0.036832
25,25,-9197520001674928649,i,13,2.008451,7.777776,0.443794,0,13,0,23.333000,0.224603,0.173000,0.025926
26,26,-9197520001674928649,r,27,1.959830,11.698710,0.360699,2,25,0,23.490815,0.258119,0.132370,0.040660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355173,355173,9105106590962218675,z,2,2.978428,8.967382,2.994477,0,2,0,23.766500,0.754483,0.477000,0.384666
355174,355174,9105640718324447285,g,24,1.696119,11.698693,0.416774,1,23,0,22.921167,0.348664,0.070708,0.025199
355195,355195,9134047254820796874,i,37,0.302818,11.991703,0.315916,5,32,0,23.223108,0.419548,0.168649,0.061941
355214,355214,9157470808996106463,g,38,1.169140,11.460404,0.270823,7,31,0,22.997263,0.248220,0.081526,0.037629
